<a href="https://colab.research.google.com/github/andrewpannn/MNIST/blob/main/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from keras.datasets import mnist
import math

def sigmoid(z):
  return 1 / (1 + np.exp(-z))

def sigmoidPrime(z):
  return np.exp(-z) / np.power(1 + np.exp(-z), 2)

(train_x, train_y), (test_x, test_y) = mnist.load_data()

#reshapes the array from 60000, 28, 28 to 60000, 784
newdata = np.reshape(train_x, (60000, 784))
newdata = np.multiply(1/255, newdata)  # scales down the values in the matrix

#turns the train_y array into a 60000 * 10 array with zeros filling the unwanted answer and a "1" for the right answer
correct = np.zeros((60000,10))
for x in train_y:
  correct[x][train_y[x]] = 1

n = 2  # learning rate

In [ ]:
class network:
  def __init__(self, layers):
    self.layers = layers
    self.layerNum = len(layers)
    self.biases = [np.random.randn(x) for x in layers[1:]]  # biases[0] is the biases for the neurons on layer 1
    self.weights = [np.random.randn(layers[x], layers[x-1]) for x in range(1, len(layers))]  # makes a matrix for weights. weights[0] is the weights from layer 0 to layer 1

thisNetwork = network([784,10,10])

In [ ]:
# forward prop

# creating the input layer
activation = [np.zeros(x) for x in thisNetwork.layers]
z = [np.zeros(x) for x in thisNetwork.layers]  # z is the matrix for the weighted inputs
activation[0] = newdata[0]


# forward prop for all layers except for input layer
def forwardProp():
  for x in range(0, thisNetwork.layerNum - 1):
    for y in range(10):  #range(number of elements in thisNetwork.layers[x])
      z[x+1][y] = np.dot(activation[x], thisNetwork.weights[x][y]) + thisNetwork.biases[x][y]
      activation[x+1][y] = sigmoid(z[x+1][y])  #propagates forward the weighted input
forwardProp()

In [ ]:
#cost function - compares result with intended result. Not actually necessary
def returnCost(result, indexNo):
    cost = np.subtract(result, correct[indexNo])
    cost = np.power(cost, 2)
    cost = 0.5*np.sum(cost)
    return cost

In [ ]:
#backprop
error = [np.zeros(x) for x in thisNetwork.layers[1:]]  # repesents the error of each particular neuron. error[0] is error of 2nd layer
#calculate error in final layer
def returnFinalError(result, z, correct):
    finGrad = np.subtract(result, correct)  # finds the rate of change of the cost with respect to final activation
    finError = np.multiply(finGrad, sigmoidPrime(z))
    return finError
#error[1] = returnFinalError(activation[2], z[2], correct[0])

In [ ]:
#calculate error in 2nd layer
def returnLayerError(error, z):
    newWeights = np.transpose(thisNetwork.weights[1])
    for x in range(10):
      weightedError = np.dot(error, newWeights[x])  # multiplying error by the transposed weight matrix
    newError = np.multiply(weightedError, sigmoidPrime(z))
    return newError

#error[0] = returnLayerError(error[1], z[1])

In [ ]:
#gradient descent 
print(returnCost(activation[2],0))
#update biases
def updateBiases(errorB):
    errorB = np.multiply(1/n, errorB)
    thisNetwork.biases = np.subtract(thisNetwork.biases, error)  # subtracts the error of each neuron from the bias matrix, elementwise
updateBiases(error)

1.2964728024384897


In [ ]:
#update weights  
def updateWeights():
    weightGrad = [np.zeros(w.shape) for w in thisNetwork.weights]
    for x in range(0, 1):
      for y in range(0, 10):
        weightGrad[x][y] = np.multiply(activation[x], error[x][y]) #  applies the transposed activation matrix to the error

      thisNetwork.weights[x] = np.subtract(thisNetwork.weights[x], 0.1 * weightGrad[x])  

updateWeights()

In [ ]:
for x in range(0, thisNetwork.layerNum - 1):
  for y in range(10):  #range(number of elements in thisNetwork.layers[x])
    z[x+1][y] = np.dot(activation[x], thisNetwork.weights[x][y]) + thisNetwork.biases[x][y]
    activation[x+1][y] = sigmoid(z[x+1][y])  #propagates forward the weighted input

print(returnCost(activation[2], 0))


1.2964728024384897


In [ ]:

thisNetwork = network([784,10,10])


# creating the input layer
activation = [np.zeros(x) for x in thisNetwork.layers]
z = [np.zeros(x) for x in thisNetwork.layers]  # z is the matrix for the weighted inputs
activation[0] = newdata[0]

for x in range(0, thisNetwork.layerNum - 1):
  for y in range(10):  #range(number of elements in thisNetwork.layers[x])
    z[x+1][y] = np.dot(activation[x], thisNetwork.weights[x][y]) + thisNetwork.biases[x][y]
    activation[x+1][y] = sigmoid(z[x+1][y])  #propagates forward the weighted input
print("pre gradient descent: ", returnCost(activation[2], 0))

#error
error = [np.zeros(x) for x in thisNetwork.layers[1:]]
error[1] = returnFinalError(activation[2], z[2], correct[0])
error[0] = returnLayerError(error[1], z[1])
  
#grad descent
updateBiases(error)
updateWeights()
print("post gradient descent: ", returnCost(activation[2], 0))


pre gradient descent:  0.7730822667814958
post gradient descent:  0.7730822667814958
